In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')

In [26]:
import warnings; warnings.simplefilter('ignore')

In [27]:
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=['user_id','item_id','rating','timestamp'])
movies = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', names=[
    'item_id','title','release_date','video_release_date','IMDb_URL','unknown',
    'Action','Adventure','Animation','Children','Comedy','Crime','Documentary',
    'Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
    'Thriller','War','Western'
])
users = pd.read_csv("ml-100k/u.user", sep="|", names=["user_id", "age", "gender", "occupation", "zip_code"])

In [28]:
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [29]:
user_item_matrix = ratings.pivot(index='user_id',columns='item_id',values='rating')
user_item_matrix.fillna(0,inplace=True)
user_item_matrix.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
interactions_matrix = pd.crosstab(ratings['user_id'],ratings['item_id']).head()
interactions_matrix

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


SVD implemntaion :

this is just a linear algerbra 

In [31]:
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds

preparing latent factors

In [39]:
R = user_item_matrix.values
user_ids = user_item_matrix.index
movie_ids = user_item_matrix.columns

In [40]:
k = 50  
U, sigma, Vt = svds(R, k=k)
sigma = np.diag(sigma)

creating prediction matrix

In [41]:
predicted_ratings = np.dot((np.dot(U,sigma)),Vt)
R_pred_df = pd.DataFrame(predicted_ratings, index=user_ids, columns=movie_ids)

In [ ]:
def recommend_movies_svd(user_id, R_df, R_pred_df, movies, n=10):


    user_ratings = R_df.loc[user_id]
    preds = R_pred_df.loc[user_id]
    
    
    already_rated = user_ratings[user_ratings > 0].index
    preds = preds.drop(already_rated)
    
    top_movies = preds.sort_values(ascending=False).head(n).index
    
    return movies[movies['item_id'].isin(top_movies)][['item_id', 'title']]

recommendations = recommend_movies_svd(5, user_item_matrix, R_pred_df, movies, n=10)
recommendations

,item_id,title
27,28,Apollo 13 (1995)
46,47,Ed Wood (1994)
55,56,Pulp Fiction (1994)
113,114,Wallace & Gromit: The Best of Aardman Animatio...
151,152,Sleeper (1973)
174,175,Brazil (1985)
183,184,Army of Darkness (1993)
194,195,"Terminator, The (1984)"
264,265,"Hunt for Red October, The (1990)"
745,746,Real Genius (1985)
